In [1]:
# !pip install PdfFileReader
from PyPDF2 import PdfFileReader
import re

In [16]:
def extract_information(pdf_path):
    pdf_data = ""
    with open(pdf_path, 'rb') as f:
        pdf = PdfFileReader(f)
        information = pdf.getDocumentInfo()
        number_of_pages = pdf.getNumPages()
        ifrs_data = []
        merged_content = ""
        start_flag = 0
        pattern = "(.*?)\s([1-9][0-9][0-9]|[1-9][0-9]|[0-9]\s)"
        for page_n in range(number_of_pages):
            page = pdf.pages[page_n]
            data = (page.extract_text() + "\n\n").lstrip()
            content_start = "OBJECTIVE"
            content_end = "APPENDICES"
            content_head_start = "INTERNATIONAL FINANCIAL REPORTING STANDARD 15 REVENUE FROM CONTRACTS WITH CUSTOMERS".replace(" ", "(\s|\n)")
            content_head_end = "Appendix A"
            if data.startswith("CONTENT"):
                heads = data[data.index(content_start): data.index(content_end)].replace("\n", " ").replace("  ", " ")
                match = re.findall(pattern, heads)
                prev_para = 1
                for head, para_num in match:
                    para_num = int(para_num.strip())
#                     if prev_para != para_num:
#                         prev_tag = ifrs_data[-1]["Tag"]
#                         for i in range(prev_para, para_num):
#                             ifrs_data.append({
#                                 "Paragraph": i,
#                                 "Tag": prev_tag
#                             })
#                         prev_para = para_num
                    head = head.strip()
                    if head.isupper():
                        head = head.capitalize()
                    ifrs_data.append({
                        "Paragraph": para_num,
                        "Tag": head
                    })
#                 return ifrs_data
#                     prev_para += 1
            if re.match(content_head_end, data, re.I):
                start_flag = 0
                merged_content += "\n" + content_head_end
            elif re.match(content_head_start, data, re.I):
                start_flag = 1
                merged_content = ""
            if start_flag:
                ending_dirt_text_cp_pattern = "(A\d+ © IFRS Foundation|© IFRS Foundation A\d+)"
                ending_dirt_text_num_pattern = "(\d+\n)*(\d+IFRS 15\s\n)$"
                if re.search(ending_dirt_text_cp_pattern, data, re.I):
                    data = re.sub(ending_dirt_text_cp_pattern, "", data)
                if re.search(ending_dirt_text_num_pattern, data, re.I):
                    data = re.sub(ending_dirt_text_num_pattern, "", data)
                merged_content += data
        merged_content = merged_content.replace("\n", "$$$")
        for i in range(len(ifrs_data)):
            if i == len(ifrs_data) - 1:
                match_paragraph_pattern =  "\$\$\$" + ifrs_data[i]["Tag"] + "\$\$\$" + "(.*?)" + "\$\$\$" + content_head_end + "\$\$\$"
                match = re.search(match_paragraph_pattern, merged_content)
            else:
                match_paragraph_pattern = "\$\$\$" + ifrs_data[i]["Tag"] + "\$\$\$" + "(.*?)" + "\$\$\$" + ifrs_data[i+1]["Tag"] + "\$\$\$"
                match = re.search(match_paragraph_pattern, merged_content)
            if match and match.group(1).strip():
                ifrs_data[i]["data"] = match.group(1).replace("$$$", " ").strip().replace("  ", " ").replace(" .", ".").replace(" -", " - ")
#         print(merged_content)
#         print(ifrs_data)
        for ifrs in ifrs_data:
            print("-" if "data" in ifrs else str(ifrs['Paragraph']) + "-" + ifrs['Tag'] + "-Missing!")
#        Para missing data: 91, 95, 113, 123, 127
    return ifrs_data

In [17]:
file = "IFRS15.pdf"
x = extract_information(file)

-
-
-
9-Recognition-Missing!
-
-
-
-
-
-
47-Determining the transaction price-Missing!
73-Allocating the transaction price to performance obligations-Missing!
-
91-Contract costs-Missing!
91-Incremental costs of obtaining a contract-Missing!
95-Costs to fulfil a contract-Missing!
-
-
-
113-Contracts with customers-Missing!
123-Significant judgements in the application of this Standard-Missing!
127-Assets recognised from the costs to obtain or fulfil a contract with a customer-Missing!
129-Practical expedients-Missing!


In [8]:
x

'International Financial Reporting Standard 15$$$Revenue from Contracts with Customers$$$Objective$$$The objective of this Standard is to establish the principles that an entity$$$shall apply to report useful information to users of financial  statements$$$about the nature, amount, timing and uncertainty of revenue  and cash$$$flows  arising from a contract  with a customer .$$$Meeting the objective$$$To meet the objective in paragraph 1, the core principle of this Standard is$$$that an entity shall recognise revenue  to depict the transfer of promised goods$$$or services to customers  in an amount that reflects the consideration to which$$$the entity expects to be entitled in exchange for those goods or services.$$$An entity shall consider the terms of the contract  and all relevant facts and$$$circumstances when applying this Standard. An entity shall apply this$$$Standard, including the use of any practical expedients, consistently to$$$contracts with similar characteristics and in 

In [4]:
x[2]

't'

In [5]:
x[2]['data']

TypeError: string indices must be integers

In [6]:
from tika import parser # pip install tika

raw = parser.from_file(file)
print(raw['content'])

2022-11-10 16:48:04,538 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to C:\Users\UIF13879\AppData\Local\Temp\tika-server.jar.


URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [7]:
import platform
from tempfile import TemporaryDirectory
from pathlib import Path
 
import pytesseract
from pdf2image import convert_from_path
from PIL import Image

In [ ]:
# Requires Python 3.6 or higher due to f-strings

# Import libraries
import platform
from tempfile import TemporaryDirectory
from pathlib import Path

import pytesseract
from pdf2image import convert_from_path
from PIL import Image


if platform.system() == "Windows":
    pytesseract.pytesseract.tesseract_cmd = (r"C:\Users\UIF13879\AppData\Local\Tesseract-OCR\tesseract.exe")
    path_to_poppler_exe = Path(r"C:\Users\UIF13879\Downloads\poppler-0.68.0_x86\poppler-0.68.0\bin")
    out_directory = Path(r"~\Desktop").expanduser()
else:
    out_directory = Path("~").expanduser()

# Path of the Input pdf
PDF_file = Path(file)

# Store all the pages of the PDF in a variable
image_file_list = []

text_file = out_directory / Path("out_text.txt")

In [ ]:
with TemporaryDirectory() as tempdir:
    """
    Part #1 : Converting PDF to images
    """

    if platform.system() == "Windows":
        pdf_pages = convert_from_path(
            PDF_file, 500, poppler_path=path_to_poppler_exe
        )
    else:
        pdf_pages = convert_from_path(PDF_file, 500)

    for page_enumeration, page in enumerate(pdf_pages, start=1):
        filename = f"{tempdir}\page_{page_enumeration:03}.jpg"
        page.save(filename, "JPEG")
        image_file_list.append(filename)

    """
    Part #2 - Recognizing text from the images using OCR
    """

    with open(text_file, "a") as output_file:
        for image_file in image_file_list:
            text = str(((pytesseract.image_to_string(Image.open(image_file)))))
            text = text.replace("-\n", "")
            output_file.write(text)

In [ ]:
text_file

In [18]:
tags = [para['Tag'] for para in x]
tags

['Objective',
 'Meeting the objective',
 'Scope',
 'Recognition',
 'Identifying the contract',
 'Combination of contracts',
 'Contract modifications',
 'Identifying performance obligations',
 'Satisfaction of performance obligations',
 'Measurement',
 'Determining the transaction price',
 'Allocating the transaction price to performance obligations',
 'Changes in the transaction price',
 'Contract costs',
 'Incremental costs of obtaining a contract',
 'Costs to fulfil a contract',
 'Amortisation and impairment',
 'Presentation',
 'Disclosure',
 'Contracts with customers',
 'Significant judgements in the application of this Standard',
 'Assets recognised from the costs to obtain or fulfil a contract with a customer',
 'Practical expedients']

In [19]:
# Remove all hyperlinks from the word doc, else it will affect the words in the para
# Ctrl+A to select all text. Press Ctrl+Shift+F9.

In [20]:
file = "IFRS15.docx"

In [21]:
import docx

In [22]:
doc = docx.Document(file)

In [23]:
for sec in doc.sections:
    print(sec.start_type)

CONTINUOUS (0)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
CONTINUOUS (0)
CONTINUOUS (0)
CONTINUOUS (0)
NEW_PAGE (2)
NEW_PAGE (2)
NEW_PAGE (2)
CONTINUOUS (0)
NEW_PAGE (2)
CONTINUOUS (0)
NEW_PAGE (2)
CONTINUOUS (0)
NEW_PAGE (2)
CONTINUOUS (0)
NEW_PAGE (2)
CONTINUOUS (0)
NEW_PAGE (2)
CONTINUOUS (0)
NEW_PAGE (2)
CONTINUOUS (0)
NEW_PAGE (2)
CONTINUOUS (0)
NEW_PAGE (2)
CONTINUOUS (0)
NEW_PAGE (2)
CONTINUOUS (0)
NEW_PAGE (2)
CONTINUOUS (0)
NEW_PAGE (2)
CONTINUOUS (0)
NEW_PAGE (2)
CONTINUOUS (0)
NEW_PAGE (2)
CONTINUOUS (0)
NEW_PAGE (2)
CONTINUOUS (0)
NEW_PAGE (2)
CONTINUOUS (0)
NEW_PAGE (2)
CONTINUOUS 

In [24]:
all_paras = doc.paragraphs
len(all_paras)

1908

In [25]:
for para in all_paras:
    if para.text in tags:
        print("\n----------------------------New Paragraph--------------------------------------\n")
        print(para.style, para.text)
    elif para.text != "":
        print("-------------------")
        print(para.style, para.text)
#         for run in para.runs:
#         print(para.style.character_style)

-------------------
_ParagraphStyle('Normal') id: 3005759016288 IFRS 15
-------------------
_ParagraphStyle('Title') id: 3005759020320 Revenue from Contracts with Customers
-------------------
_ParagraphStyle('Normal') id: 3005759016432 In  April   2001   the   International   Accounting   Standards   Board   (Board)   adopted   IAS 11 Construction Contracts and IAS 18 Revenue, both of which had originally been issued by the International Accounting Standards Committee (IASC) in December 1993. IAS 18 replaced a previous version: Revenue Recognition (issued in December 1982). IAS 11 replaced parts of IAS 11 Accounting for Construction Contracts (issued in March 1979).
-------------------
_ParagraphStyle('Body Text') id: 3005759026176 In December 2001 the Board issued SIC‑31 Revenue—Barter Transactions Involving Advertising Services. The Interpretation was originally developed by the Standards Interpretations Committee of the IASC to determine the circumstances in which a seller of adver

_ParagraphStyle('Heading 4') id: 3005759809824 Constraining estimates of variable consideration
-------------------
_ParagraphStyle('List Paragraph') id: 3005759021088 An entity shall include in the transaction price some or all of an amount of variable consideration estimated in accordance with paragraph 53 only to the extent that it is highly probable that a significant reversal in the amount of cumulative revenue recognised will not occur when the uncertainty associated with the variable consideration is subsequently resolved.
-------------------
_ParagraphStyle('List Paragraph') id: 3005759013744 In assessing whether it is highly probable that a significant reversal in the  amount of cumulative revenue recognised will not occur once the uncertainty related to the variable consideration is subsequently resolved, an entity shall consider both the likelihood and the magnitude of the revenue reversal. Factors that could increase the likelihood or the magnitude of a revenue reversal inc

_ParagraphStyle('Body Text') id: 3005759023152 
The price at which an entity would sell a promised good or  service separately to a customer.
-------------------
_ParagraphStyle('Body Text') id: 3005759020656 The amount of consideration to which an entity expects to be entitled in exchange for transferring promised goods or services to a customer, excluding amounts collected on behalf of third parties.
-------------------
_ParagraphStyle('Heading 1') id: 3005759020368 Appendix B Application Guidance
-------------------
_ParagraphStyle('Normal') id: 3005759020656 This appendix is an integral part of the Standard. It describes the application of paragraphs 1–129 and has the same authority as the other parts of the Standard.
-------------------
_ParagraphStyle('Body Text') id: 3005759023152 B1	This application guidance is organised into the following categories:
-------------------
_ParagraphStyle('List Paragraph') id: 3005759020704 performance obligations satisfied over time (paragraphs 

_ParagraphStyle('Body Text') id: 3005759024592 
If a customer has the option to acquire an additional good or service at a price that would reflect the stand-alone selling price for that good or service, that option does not provide the customer with a material right even if the option can be exercised only by entering into a previous contract. In those cases, the entity has made a marketing offer that it shall account for in accordance with this Standard only when the customer exercises the option to purchase the additional goods or services.
-------------------
_ParagraphStyle('Body Text') id: 3005759014848 Paragraph 74 requires an entity to allocate the transaction price to performance obligations on a relative stand-alone selling price basis. If the stand-alone selling price for a customer’s option to acquire additional goods or services is not directly observable, an entity shall estimate it. That estimate shall reflect the discount that the customer would obtain when exercising t

In [26]:
from docx.api import Document

In [27]:
document = Document(file)

In [28]:
ifrs_data = []

In [29]:
for para in all_paras:
    if para.style.name.startswith('Heading 2'):
        print(para.text)
    if para.style.name.startswith('Heading 1') and para.text == "Appendix A Defined terms":
        break

Objective
Meeting the objective
Scope
Recognition
Identifying the contract
Combination of contracts
Contract modifications
Identifying performance obligations
Satisfaction of performance obligations
Measurement
Determining the transaction price
Allocating the transaction price to performance obligations
Changes in the transaction price
Contract costs
Incremental costs of obtaining a contract
Costs to fulfil a contract
Amortisation and impairment
Presentation
Disclosure
Contracts with customers
Significant judgements in the application of this Standard
Assets recognised from the costs to obtain or fulfil a contract with a customer
Practical expedients
